Basic idea:
decorator for init functions so that constructors with a lot of arguments can be initialized from a configuration file.




In [113]:
import configparser
import inspect
import os

def args_from_configfile(func):
    """A decorator that enables to retrieve function arguments from config files.
    Instead of passing all arguments, you can also pass a keyword 'cfg_file' 
    and a keyword 'section'.
    The function is then called with the parameters specified in the section of
    a config file. 
    
    When using this decorator on constructors, the class is created from cfg_file.
    
    Note that the keyword "cfg_file" overrides all other function
    arguments. 
    Note also that all arguments specified config file's section are
    passed as function parameters.
    """
    args = inspect.getfullargspec(func)[0]
    print(args)
    wrapper = None
    if len(args) > 0 and args[0] == "self":
        def method_wrapper(*args,**kwargs):     
            if "cfg_file" in kwargs:

                cfg_file = kwargs["cfg_file"]
                assert "section" in kwargs
                section = kwargs["section"]
                assert os.path.isfile(cfg_file)

                cfg = configparser.ConfigParser()
                cfg.read(cfg_file)
                assert cfg.has_section(section)
                options_dict = {}
                for opt in cfg.options(section):
                    options_dict[opt] = cfg.get(section,opt)
                return func(self=args[0],**options_dict)
            else:
                return func(*args,**kwargs)
        wrapper = method_wrapper
    else:
        def function_wrapper(*args,**kwargs):     
            if "cfg_file" in kwargs:

                cfg_file = kwargs["cfg_file"]
                assert "section" in kwargs
                section = kwargs["section"]
                assert os.path.isfile(cfg_file)

                cfg = configparser.ConfigParser()
                cfg.read(cfg_file)
                assert cfg.has_section(section)
                options_dict = {}
                for opt in cfg.options(section):
                    options_dict[opt] = cfg.get(section,opt)
                return func(**options_dict)
            else:
                return func(*args,**kwargs)
        wrapper = function_wrapper    
    return wrapper

In [114]:
class A:
    @args_from_configfile
    def __init__(self, arg1, arg2, arg3, arg4=None, arg5=None):
        print(arg1,arg2,arg3,arg4,arg5)
        
@args_from_configfile
def test_function(arg1,arg2,arg3):
    print("test function with", arg1, arg2, arg3)

['self', 'arg1', 'arg2', 'arg3', 'arg4', 'arg5']
['arg1', 'arg2', 'arg3']


In [115]:
a = A(1,2,3)
b = A(1,2,3,4,cfg_file="test.cfg",section="test1")
c = A(cfg_file="test.cfg",section="test2")
test_function(cfg_file="test.cfg",section="test2")

1 2 3 None None
10 11 12 13 14
100 101 102 None None
test function with 100 101 102
